# Iteration 5: cancelled electives (KPI)

## Plain English Summary

In this iteration we add in teh first basic performance measure calculation.

1. Number of cancelled elective operations.

These calculations only use statistics collected in the results collection period.


## Notebook imports

In [1]:
from IPython.display import Markdown, display

import numpy as np
import math

## LLM Prompt

In [2]:
display(Markdown("inputs/iteration_5.md"))

modify the code as follows:

The code should calculate performance measures at the end of the simulation run.  

The following measures should be calculated.

1. The number of elective operations cancelled. 

The code should only collect statistics on this performance measure if the simulation time has exceeded the warm-up period.

After the performance measures are calculated the results should outputted in a pandas dataframe.

Show all of the model code: this includes all patient generator functions in the CCModel class and all code in the Experiment class.

## Date and time of input to LLM

> 1540 30th November 2023 

## Observations on the LLM Output

* The code appears to correctly add cancelled operations to the count if the warm-up period is completed.  I.e.

```python
if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
    print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
    if self.env.now > self.experiment.warm_up_period:
        self.cancelled_elective_operations += 1
```

* The code checks for warm-up period completion at the end of the run before printing out the statistics.  This was unexpected, but makes sense on one level as the no statistics will have been collection if the simulation has run less than the warm-up period.

## LLM Output 1:

In this modified code, the CCUModel class calculates the number of elective operations cancelled at the end of the simulation run. If the simulation time has exceeded the warm-up period, statistics on the performance measure are collected, and the results are outputted in a pandas dataframe.

In [3]:
import simpy
import numpy as np
import pandas as pd

class Experiment:
    def __init__(self, 
                 interarrival_means=[22.72, 26.0, 37.0, 47.2, 575.0, 17.91], 
                 stay_distributions=[(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67), 57.34], 
                 elective_treatment_mean=57.34, 
                 num_critical_care_beds=24, 
                 intensive_cleaning_duration=5, 
                 warm_up_period=30*24, 
                 results_collection_period=12*30*24):
        self.interarrival_means = interarrival_means
        self.stay_distributions = stay_distributions
        self.elective_treatment_mean = elective_treatment_mean
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.warm_up_period = warm_up_period
        self.results_collection_period = results_collection_period

class CCUModel:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_count = 0
        self.cancelled_elective_operations = 0
        self.critical_care_beds = simpy.Resource(env, capacity=experiment.num_critical_care_beds)

    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[0]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[0]))

    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[1]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[1]))

    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[2]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[2]))

    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[3]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[3]))

    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[4]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[4]))

    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(self.experiment.interarrival_means[5], 3.16))
            self.patient_count += 1
            print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
                if self.env.now > self.experiment.warm_up_period:
                    self.cancelled_elective_operations += 1
            else:
                self.env.process(self.elective_surgery_process(self.experiment.elective_treatment_mean))

    def unplanned_admission(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def elective_surgery_process(self, treatment_mean):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(treatment_mean)
            yield self.env.timeout(treatment_time)
            print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def run(self):
        self.env.process(self.patient_arrival_AE())
        self.env.process(self.patient_arrival_wards())
        self.env.process(self.patient_arrival_surgery())
        self.env.process(self.patient_arrival_other_hospitals())
        self.env.process(self.patient_arrival_X_ray())
        self.env.process(self.patient_arrival_elective_surgery())
        self.env.run(until=self.experiment.results_collection_period + self.experiment.warm_up_period)
        if self.env.now > self.experiment.warm_up_period:
            df = pd.DataFrame({'Cancelled Elective Operations': [self.cancelled_elective_operations]})
            print(df)


In [4]:
# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the Experiment class and the CCU model, then run the simulation
experiment = Experiment()
ccu_model = CCUModel(env, experiment)
ccu_model.run()

Patient 1 arrived from other hospitals at time 4.70855162454995
Patient 1 admitted to critical care bed at time 4.70855162454995
Patient 2 arrived from Emergency surgery at time 9.03722190430486
Patient 2 admitted to critical care bed at time 9.03722190430486
Patient 3 arrived from Accident and Emergency at time 11.076166452812041
Patient 3 admitted to critical care bed at time 11.076166452812041
Elective surgery patient 4 arrived at time 20.356724814339863
Elective surgery patient 4 admitted to critical care bed at time 20.356724814339863
Patient 5 arrived from the Wards at time 21.862743031291355
Patient 5 admitted to critical care bed at time 21.862743031291355
Patient 6 arrived from other hospitals at time 26.56587992035304
Patient 6 admitted to critical care bed at time 26.56587992035304
Patient 7 arrived from the Wards at time 30.39614440494274
Patient 7 admitted to critical care bed at time 30.39614440494274
Patient 7 discharged from critical care bed at time 33.503921326347125


## Testing

Here we do a number of regression tests to check that previous functionality is not affected by the `Experiment` class. We also update the test of run length splitting into two tests varying a results collection period and warm-up period.

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Elective."
* **Different results collection period**: The results collection period of the model is set to 10 hours.
    * Expected result: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
    * Actual result **(PASS)**: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
* **Extreme value test 3**: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result **(PASS)**: queues form after first arrival. 
* **Extreme value test 4**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result **(PASS)**: after 24 arrivals queues form and no patients are admitted.
* **Vary warm up period**: Vary the warm-up period while holding the results collection period constant.
    * Expected result: Run length is equal to the `results_collection_period`
    * Actual result **(PASS)**: Run length is equal to the `results_collection_period`

In [5]:
# The extreme value
M = 10_000_000

### Test 1: Extreme value test 1

PASS

In [6]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCUModel(env, experiment)
    ccu_model.run()
    return ccu_model

In [7]:
def extreme_value_test_1(extreme_value=M):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time is
    set to $M$ a very large number.
    Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    There are no elective cancellations (as no electives arrive)
    '''
    mod_interarrival_means = [22.72] + [extreme_value] * 5 
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [8]:
extreme_value_test_1()

Patient 1 arrived from Accident and Emergency at time 9.44881106107919
Patient 1 admitted to critical care bed at time 9.44881106107919
Patient 2 arrived from Accident and Emergency at time 23.301927828274927
Patient 2 admitted to critical care bed at time 23.301927828274927
Patient 2 discharged from critical care bed at time 80.34270170445123
Critical care bed is available for next patient at time 85.34270170445123
Patient 3 arrived from Accident and Emergency at time 88.79102946399742
Patient 3 admitted to critical care bed at time 88.79102946399742
Patient 3 discharged from critical care bed at time 92.48841526791017
Critical care bed is available for next patient at time 97.48841526791017
Patient 4 arrived from Accident and Emergency at time 98.21078439838415
Patient 4 admitted to critical care bed at time 98.21078439838415
Patient 5 arrived from Accident and Emergency at time 99.27333697387841
Patient 5 admitted to critical care bed at time 99.27333697387841
Patient 5 discharged f

### Test 2: Extreme value test 2

PASS

In [9]:
def extreme_value_test_2(extreme_value=M):
    '''
    Extreme value test 2: 
    
    All unplanned admissions have their inter-arrival time is
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    mod_interarrival_means=[extreme_value] * 5 + [17.91]
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [10]:
extreme_value_test_2()

Elective surgery patient 1 arrived at time 20.097184991077235
Elective surgery patient 1 admitted to critical care bed at time 20.097184991077235
Elective surgery patient 1 discharged from critical care bed at time 33.04513627894859
Critical care bed is available for next patient at time 38.04513627894859
Elective surgery patient 2 arrived at time 38.797210009078114
Elective surgery patient 2 admitted to critical care bed at time 38.797210009078114
Elective surgery patient 3 arrived at time 55.64248920017569
Elective surgery patient 3 admitted to critical care bed at time 55.64248920017569
Elective surgery patient 3 discharged from critical care bed at time 57.65021336801837
Critical care bed is available for next patient at time 62.65021336801837
Elective surgery patient 4 arrived at time 72.94076065308857
Elective surgery patient 4 admitted to critical care bed at time 72.94076065308857
Elective surgery patient 4 discharged from critical care bed at time 73.10714419925975
Critical ca

### Test 3: Different run length

PASS

In [11]:
def test_results_collection_period(new_collect_period):
    '''
    Vary the results collection period while holding the 
    warm-up period constant.

    Expected result Run length should be no longer than `new_run_length`
    '''
    experiment = Experiment(results_collection_period=new_collect_period)
    model = run_test(experiment)
    return model.env.now

In [12]:
test_results_collection_period(10)

Patient 1 arrived from Emergency surgery at time 3.9532747388672442
Patient 1 admitted to critical care bed at time 3.9532747388672442
Patient 2 arrived from Emergency surgery at time 4.721603205504198
Patient 2 admitted to critical care bed at time 4.721603205504198
Patient 3 arrived from Accident and Emergency at time 11.678580371138777
Patient 3 admitted to critical care bed at time 11.678580371138777
Patient 3 discharged from critical care bed at time 13.900165136386757
Critical care bed is available for next patient at time 18.900165136386757
Elective surgery patient 4 arrived at time 20.86027866268697
Elective surgery patient 4 admitted to critical care bed at time 20.86027866268697
Patient 5 arrived from Accident and Emergency at time 42.89947628150151
Patient 5 admitted to critical care bed at time 42.89947628150151
Elective surgery patient 6 arrived at time 43.2039107691007
Elective surgery patient 6 admitted to critical care bed at time 43.2039107691007
Elective surgery patie

730

### Test 4: Extreme value test 3

PASS.

In [13]:
def extreme_value_test_3(critical_care_beds=1):
    '''
    Extreme value test 3: 
    
    Critical care beds set to 1 or parameter
    
    Expected result: when critical_care_beds=1 queues form after first arrival.
    Cancellations also begin after 1st arrival.
    '''
    experiment = Experiment(num_critical_care_beds=critical_care_beds)
    run_test(experiment)

In [14]:
extreme_value_test_3()

Patient 1 arrived from the Wards at time 6.436729687898734
Patient 1 admitted to critical care bed at time 6.436729687898734
Patient 2 arrived from Accident and Emergency at time 8.104481101210162
Patient 3 arrived from Accident and Emergency at time 10.774210209205338
Patient 4 arrived from the Wards at time 11.349377300257611
Elective surgery patient 5 arrived at time 18.236029238103217
Elective surgery for patient 5 cancelled due to no available critical care beds at time 18.236029238103217
Patient 6 arrived from Emergency surgery at time 21.2516842893563
Patient 7 arrived from Accident and Emergency at time 26.091893363267708
Patient 8 arrived from Emergency surgery at time 32.76958581132794
Patient 9 arrived from other hospitals at time 34.258594955774896
Patient 10 arrived from the Wards at time 36.45856381957576
Patient 11 arrived from Accident and Emergency at time 38.817613903928326
Elective surgery patient 12 arrived at time 39.98863881175796
Elective surgery for patient 12 c

### Test 5: Extreme value test 4

PASS

In [15]:
def extreme_value_test_4(extreme_value=M):
    '''
    Extreme value test 4: 
    
    means of stay_distributions set to $M$ a very large number.
    Expected result: after 24 arrivals queues form and no patients are admitted.
    '''
    mod_stay_distributions=[(M, 267.51), 
                            (M, 276.54), 
                            (M, 218.02), 
                            (M, 457.67), 
                            (M, 108.67), 
                            57.34] 
    experiment = Experiment(stay_distributions=mod_stay_distributions, 
                            elective_treatment_mean=M)
    run_test(experiment)

In [16]:
extreme_value_test_4()

Patient 1 arrived from Accident and Emergency at time 1.3070513091972666
Patient 1 admitted to critical care bed at time 1.3070513091972666
Patient 2 arrived from the Wards at time 2.8184355985844842
Patient 2 admitted to critical care bed at time 2.8184355985844842
Patient 3 arrived from other hospitals at time 18.17910140307242
Patient 3 admitted to critical care bed at time 18.17910140307242
Elective surgery patient 4 arrived at time 20.465114899693667
Elective surgery patient 4 admitted to critical care bed at time 20.465114899693667
Patient 5 arrived from Accident and Emergency at time 32.17277190680894
Patient 5 admitted to critical care bed at time 32.17277190680894
Patient 6 arrived from Accident and Emergency at time 33.23459639465133
Patient 6 admitted to critical care bed at time 33.23459639465133
Patient 7 arrived from Accident and Emergency at time 36.58902716005336
Patient 7 admitted to critical care bed at time 36.58902716005336
Elective surgery patient 8 arrived at time

### Test 6: Vary Warm-up parameter

PASS

In [17]:
def test_warmup_period(new_warmup_period):
    '''
    Vary the warm-up period while holding the 
    results collection period constant.

    Expected result Run length should be no longer than results_collection_period
    '''
    experiment = Experiment(warm_up_period=new_warmup_period)
    model = run_test(experiment)
    return model.env.now, experiment.results_collection_period

In [18]:
test_warmup_period(0)

Patient 1 arrived from the Wards at time 2.8381129075182336
Patient 1 admitted to critical care bed at time 2.8381129075182336
Elective surgery patient 2 arrived at time 12.158584177370354
Elective surgery patient 2 admitted to critical care bed at time 12.158584177370354
Elective surgery patient 2 discharged from critical care bed at time 16.04509007107978
Critical care bed is available for next patient at time 21.04509007107978
Elective surgery patient 3 arrived at time 30.035273284999302
Elective surgery patient 3 admitted to critical care bed at time 30.035273284999302
Patient 4 arrived from the Wards at time 37.02036182668519
Patient 4 admitted to critical care bed at time 37.02036182668519
Patient 5 arrived from the Wards at time 40.1246057858953
Patient 5 admitted to critical care bed at time 40.1246057858953
Patient 6 arrived from Accident and Emergency at time 49.4066735980071
Patient 6 admitted to critical care bed at time 49.4066735980071
Patient 7 arrived from the Wards at 

(8640, 8640)